### Introduzione

I **sistemi di raccomandazione** sono dei sistemi software che permettono di predire le preferenze di un utente basandoci sulle preferenze espresse dall'utente in passato.  
A differenza dei sistemi tradizionali, essi possono predire il gradimento dell'utente anche per oggetti rari, evitando il cosiddetto fenomeno **long-tail**.  
Un sistema di raccomandazione è strutturato in una matrice chiamata **matrice di utilità**, avente:  
- nelle righe, gli utenti del sistema.  
- nelle colonne, gli oggetti da valutare.  
- nelle celle, la valutazione dell'oggetto nella colonna `j` relativa all'utente nella riga `i`.  

I sistemi di raccomandazione possono diversi in due categorie:  
-  **content-based**: ad ogni utente è associato un profilo che verrà utilizzato per effettuare le predizioni sugli item.  
- **collaborative-filtering**: le predizioni sugli item vengono fatte basandoci su utenti simili (nel caso di **user-based** collaborative filtering) oppure su item simili (nel caso di **item-based** collaborative filtering).  

### Obbiettivo
Creeremo diversi sistemi di raccomandazione e ne confronteremo le prestazioni. Useremo anche un sistema personalizzato che tenga in considerazione delle data in cui è stata lasciata la valutazione.  
Useremo la versione ridotta di questo [dataset](https://grouplens.org/datasets/movielens/latest/) per generare i sistemi. Al suo interno sono presenti due tabelle `.csv`:  
- `movies.csv`: contiene l'elenco dei film, con i campi `movieId`, `title` e `genres`.  
- `ratings.csv`: contiene l'elenco delle valutazione relative agli utenti, con i campi `userId`, `movieId`, `rating`, `timestamp`. 


### Preparazione dell'ambiente
All'interno del nostro progetto, useremo diverse librerie:  
- [`pandas`](https://pandas.pydata.org/), permette di all'interno i dati all'interno .  
- [`surprise`](https://surpriselib.com/), permette la creazione e l'addestramento dei sistemi di raccomandazione in Python.  

Iniziamo leggendo dal file `movies.csv` i film e dal file `ratings.csv` le valutazioni degli utenti.
Successivamente, rimuoviamo dai dataset eventuali righe che contengono valori `NaN` poiché potrebbero compromettere il risultato finale dell'analisi.

In [1]:
import os

import pandas as pd

movie_dataset_path = os.path.join(os.getcwd(), 'movie-dataset', 'movies.csv')
rating_dataset_path = os.path.join(os.getcwd(), 'movie-dataset', 'ratings.csv')

movie_dataset = pd.read_csv(movie_dataset_path, sep=',', engine='python')
rating_dataset = pd.read_csv(rating_dataset_path, sep=',', engine='python')

movie_dataset = movie_dataset.dropna()
rating_dataset = rating_dataset.dropna()

Una volta preparati i dataset, possiamo procedere con la conversione del dataset in una struttura dati che verrà utilizzata per l'addestramento dei sistemi di raccomandazione.

In [2]:
from surprise import Dataset, Reader

elaborated_data = Dataset.load_from_df(rating_dataset[['userId', 'movieId', 'rating']], Reader(rating_scale=(0.5, 5.0)))

### Analisi dei dati
Dal dataset `ratings.csv` possiamo estrarre alcune informazioni utili per l'analisi dei dati.
In particolare, possiamo ricavare l'elenco degli utenti che hanno lasciato almeno una valutazione.
Il comportamento degli utenti può già darci una prima idea di come sarà la struttura della matrice di utilità.

Procediamo quindi ad estrarre dal dataset delle valutazione tutti gli utenti.

In [9]:
users_list = rating_dataset['userId'].unique().tolist()
users_dataset = pd.DataFrame(columns=['userId', 'mean', 'weighted_mean', 'mean_diff', 'time_diff'])

for user in users_list:
    single_ratings = rating_dataset[rating_dataset['userId'] == user]
    single_last_timestamp = single_ratings['timestamp'].max()
    single_first_timestamp = single_ratings['timestamp'].min()
    user_mean = single_ratings['rating'].mean()
    user_weighted_mean = (single_ratings['rating'] * single_ratings['timestamp']).sum() / single_ratings['timestamp'].sum()
    user_mean_diff = abs(user_mean - user_weighted_mean)
    user_time_diff = single_last_timestamp - single_first_timestamp
    users_dataset = pd.concat([users_dataset, pd.DataFrame([[user, user_mean, user_weighted_mean, user_mean_diff, user_time_diff]], columns=['userId', 'mean', 'weighted_mean', 'mean_diff', 'time_diff'], index=[user])])
    
users_dataset

,userId,mean,weighted_mean,mean_diff,time_diff
1,1,4.366379,4.366378,1.201621e-06,739163
2,2,3.948276,3.948276,1.803723e-08,505
3,3,2.435897,2.435897,1.157138e-07,970
4,4,3.555556,3.551488,4.067785e-03,62496114
5,5,3.636364,3.636364,2.212071e-08,590
...,...,...,...,...,...
606,606,3.657399,3.655863,1.536219e-03,197231731
607,607,3.786096,3.784607,1.489189e-03,34768995
608,608,3.134176,3.138498,4.322623e-03,72402187
609,609,3.270270,3.270270,1.130064e-08,278


In [4]:
import numpy as np
from surprise import AlgoBase, accuracy
from surprise.model_selection import KFold, train_test_split


def test_algorithm(algorithm: AlgoBase):
    train, test = train_test_split(elaborated_data, test_size=0.2)
    algorithm.fit(train)
    split_predictions = algorithm.test(test)
    split_predictions_measure = dict()
    split_predictions_measure['rmse'] = accuracy.rmse(split_predictions, verbose=False)
    split_predictions_measure['mae'] = accuracy.mae(split_predictions, verbose=False)
    split_predictions_measure['mse'] = accuracy.mse(split_predictions, verbose=False)
    
    kf = KFold(n_splits=5)
    cross_predictions_measure = dict()
    cross_predictions_measure['rmse'] = np.array([])
    cross_predictions_measure['mae'] = np.array([])
    cross_predictions_measure['mse'] = np.array([])
    for k_train, k_test in kf.split(elaborated_data):
        algorithm.fit(k_train)
        k_predictions = algorithm.test(k_test)
        cross_predictions_measure['rmse'] = np.append(cross_predictions_measure['rmse'], accuracy.rmse(k_predictions, verbose=False))
        cross_predictions_measure['mae'] = np.append(cross_predictions_measure['mae'], accuracy.mae(k_predictions, verbose=False))
        cross_predictions_measure['mse'] = np.append(cross_predictions_measure['mse'], accuracy.mse(k_predictions, verbose=False))
    
    return (split_predictions_measure, cross_predictions_measure)

In [5]:
from surprise import SVD, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore

algorithms = list([SVD(), KNNBaseline(k=40, verbose=False), KNNBasic(k=40, verbose=False), KNNWithMeans(k=40, verbose=False), KNNWithZScore(k=40, verbose=False)])
algorithms_results = pd.DataFrame(columns=['Algorithm', 'Split RMSE', 'Split MAE', 'Split MSE', 'Cross RMSE', 'Cross MAE', 'Cross MSE'])

for algo in algorithms:
    split_result, cross_result = test_algorithm(algo)
    result_df = pd.DataFrame({
        'Algorithm': algo.__class__.__name__, 
        'Split RMSE': split_result['rmse'], 
        'Split MAE': split_result['mae'], 
        'Split MSE': split_result['mse'], 
        'Cross RMSE': cross_result['rmse'].mean(), 
        'Cross MAE': cross_result['mae'].mean(), 
        'Cross MSE': cross_result['mse'].mean()
    }, index=[0])
    algorithms_results = pd.concat([algorithms_results, result_df], ignore_index=True)
    
algorithms_results

,Algorithm,Split RMSE,Split MAE,Split MSE,Cross RMSE,Cross MAE,Cross MSE
0,SVD,0.874306,0.671420,0.764411,0.874029,0.671270,0.763952
1,KNNBaseline,0.873887,0.669045,0.763678,0.874238,0.668292,0.764306
2,KNNBasic,0.937227,0.718864,0.878395,0.947663,0.726492,0.898076
3,KNNWithMeans,0.897727,0.684257,0.805915,0.896409,0.685266,0.803554
4,KNNWithZScore,0.898323,0.680780,0.806983,0.897047,0.680508,0.804725
